In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from random import random
import matplotlib.pyplot as plt
from utils import *
from tqdm.notebook import tqdm
import multiprocessing
from uuid import uuid4
from game_mechanics import (
    ChooseMoveCheckpoint,
    PokerEnv,
    checkpoint_model,
    choose_move_randomly,
    human_player,
    load_network,
    play_poker,
    save_network,
    State,
    to_basic_nn_input,
)

%load_ext autoreload
%autoreload 2

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class PokerPolicy(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(55, 20),
            nn.LeakyReLU(),
            nn.Linear(20,5),
        )
        
    def forward(self, data, legal_moves):
        ## legal moves is list of lists of allowed moves
        illegal = lambda legal: [move not in legal for move in range(5)]
        mask = torch.stack([torch.as_tensor(illegal(lm)) for lm in legal_moves])
        preds = self.model(data)
        preds = preds.masked_fill(mask, -float('inf'))
        return F.softmax(preds,dim=-1)

In [3]:
lr_policy = 0.005
lr_value = 0.01
gamma = 1.0

policy = PokerPolicy()
value = nn.Sequential(
    nn.Linear(55, 20),
    nn.LeakyReLU(),
    nn.Linear(20,1)
)

optimizer_policy = torch.optim.Adam(policy.parameters(), lr=lr_policy)
optimizer_value  = torch.optim.Adam(value.parameters(), lr=lr_value)

# env = CartPoleEnv()
erm = ParallelEpisodeReplayMemory(gamma=0.99, lamda=1.0)
num_episodes = 1_000_000

In [4]:
def play_poker(env):
    erm = ParallelEpisodeReplayMemory(gamma=0.99, lamda=1.0)
    episode_id = uuid4()
    state, reward, done, _ = env.reset()
    state = to_basic_nn_input(state)
    while not done:
        prev_state = state
        legal_moves = [m._value_ for m in env.game.get_legal_actions()]
        with torch.no_grad():
            probs = policy(state, [legal_moves])
            chosen_move = np.random.choice(range(0,5), p=probs[0].detach().numpy())
        state, reward, done, _ = env.step(chosen_move)
        state = to_basic_nn_input(state)
        erm.append({'state': [state], 
                    'prev_state': [prev_state], 
                    'reward': reward,
                    'done': bool(done),
                    'chosen_move': chosen_move,
                    'prob_move': probs[0][chosen_move],
                    'episode_id': episode_id})
    return erm.data

multiprocessing.cpu_count()
threads = 8

remaining_episodes = num_episodes
all_games = []
batch_size = 2000
while remaining_episodes > 0:
    print(remaining_episodes)
    pool = multiprocessing.Pool(processes=threads)
    environments = [PokerEnv(opponent_choose_move = choose_move_randomly) for _ in range(threads)]
    games = pool.map(play_poker, environments)
    all_games.append(games)
    if sum([len(game) for game in all_games]) >= batch_size:
        remaining_episodes -= batch_size
        sample = pd.concat(all_games)
        all_games = []
        
        
        # fit value function
        optimizer_value.zero_grad()
        baseline_estimates = value(sample['prev_state'])[:,0]
        loss_value = F.smooth_l1_loss(baseline_estimates, sample['discounted_rewards'][0])
        loss_value.backward()
        optimizer_value.step()
        losses_value.append(loss_value.item())

        # fit policy function
        optimizer_policy.zero_grad()
        probs = policy(sample['prev_state'])
        moves = sample['chosen_move']
        normalized_rewards = sample['discounted_rewards'][0] - baseline_estimates.detach()
        loss_policy = -(torch.log(probs[range(episode_len),moves.long()])*normalized_rewards).sum()/episode_len
        loss_policy.backward()
        losses_policy.append(loss_policy.item())
        optimizer_policy.step()

1000000


/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.data  = pd.concat([self.data, episode])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episodes[episode_id] = pd.concat([self.current_episodes[episode_id], new_row])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episodes[episode_id] = pd.concat([self.current_episodes[episode_id], new_row])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:32

1000000


/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.data  = pd.concat([self.data, episode])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episodes[episode_id] = pd.concat([self.current_episodes[episode_id], new_row])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.data  = pd.concat([self.data, episode])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:32: FutureWarning: In a future version, object-dt

/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episodes[episode_id] = pd.concat([self.current_episodes[episode_id], new_row])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episodes[episode_id] = pd.concat([self.current_episodes[episode_id], new_row])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episodes[episode_id] = pd.concat([self.current_episodes[episode_id], new_row])
/home/hristo/

...

1000000


/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episodes[episode_id] = pd.concat([self.current_episodes[episode_id], new_row])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episodes[episode_id] = pd.concat([self.current_episodes[episode_id], new_row])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.data  = pd.concat([self.data, episode])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25

/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episodes[episode_id] = pd.concat([self.current_episodes[episode_id], new_row])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episodes[episode_id] = pd.concat([self.current_episodes[episode_id], new_row])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:25: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episodes[episode_id] = pd.concat([self.current_episodes[episode_id], new_row])
/home/hristo/

KeyboardInterrupt: 

  File "/home/hristo/miniconda3/envs/cv38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/hristo/miniconda3/envs/cv38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/hristo/miniconda3/envs/cv38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hristo/miniconda3/envs/cv38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hristo/miniconda3/envs/cv38/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/hristo/miniconda3/envs/cv38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hristo/miniconda3/envs/cv38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(

In [9]:
remaining_episodes

1000000

In [63]:
erm

[]

In [54]:
erm.data

[]

In [12]:
state

State(hand=['DQ', 'C9'], public_cards=[], player_chips=2, opponent_chips=97, player_chips_remaining=101, opponent_chips_remaining=0, stage=0, legal_actions=[0, 1, 3, 4])

In [29]:
legal_moves = [m._value_ for m in env.game.get_legal_actions()]

In [30]:
legal_moves

[0, 1]

In [28]:
legal_moves[0]._value_

0

In [13]:
num_episodes = 5
for ep_num in tqdm(range(num_episodes)):
    num_steps = 0
    state, reward, done, _ = env.reset()
    state = to_basic_nn_input(state)
    while not done:
        prev_state = state
        legal_moves = [m._value_ for m in env.game.get_legal_actions()]
        with torch.no_grad():
            probs = policy(state)
            chosen_move = 4 #np.random.choice(range(0,5), p=probs.detach().numpy())
        state, reward, done, _ = env.step(chosen_move)
        state = to_basic_nn_input(state)
        erm.append({'state': [state], 
                    'prev_state': [prev_state], 
                    'reward': reward,
                    'done': done,
                    'chosen_move': chosen_move,
                    'prob_left': probs[chosen_move]})

  0%|          | 0/5 [00:00<?, ?it/s]

[<Action.FOLD: 0>, <Action.CHECK_CALL: 1>, <Action.RAISE_POT: 3>, <Action.ALL_IN: 4>]
[<Action.FOLD: 0>, <Action.CHECK_CALL: 1>, <Action.RAISE_POT: 3>, <Action.ALL_IN: 4>]
[<Action.FOLD: 0>, <Action.CHECK_CALL: 1>]


/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:23: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episode = pd.concat([self.current_episode, new_row])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:23: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.current_episode = pd.concat([self.current_episode, new_row])
/home/hristo/Delta-academy/3.2 Poker (version 2)/utils.py:35: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.data  = pd.concat([self.data, self.current_episode])


AssertionError: 4 is an illegal move

In [34]:
num_episodes

5

In [37]:
to_basic_nn_input(state)

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000, -0.9800, -0.9600, -0.0200])

In [35]:
state

State(hand=['S7', 'D5'], public_cards=[], player_chips=2, opponent_chips=4, player_chips_remaining=98, opponent_chips_remaining=96, stage=0, legal_actions=[0, 1, 2, 3, 4])

In [16]:
env.game.configure?

In [12]:
play_poker(human_player, choose_move_randomly)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Windows\\Fonts\\arial-narrow.ttf'

In [10]:
human_player?